In [1]:
import sys
sys.path.append('../')

In [2]:
from jigsaw.models.linear_models.base_model import LinearModel, KernelModel, SVRModel
from jigsaw.models.cnn_models.base_model import CnnModel
from jigsaw.models.rnn_models.base_model import RnnModel
from jigsaw.utils.tokenizer import Tokenizer
from box import Box
from jigsaw.utils.glove import load_glove
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from jigsaw.scripts.training import linear_train
from jigsaw.scripts.inference import linear_predict
from jigsaw.utils.cleaning import *
import nltk
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/alexander/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexander/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/alexander/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /Users/alexander/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
cfg = {
    'seed': 42,
    'emb_type': 'fasttext',
    'model_type': 'linear',
    'project': 'Jigsaw',
    'dataset': {
        'name': 'toxic-comment-preprocessed',
        'more_toxic': 'more_toxic',
        'less_toxic': 'less_toxic',
        'text_col': 'comment_text',
        'target_col': 'y'
    },
    'cache_features': True,
    'alpha': 1,
    'ngram_range': (2, 5),
    'kernel': 'poly',
    'gamma': 'scale',
    'degree': 3,
    'C': 1,
    'max_df': 1,
    'min_df': 1,
    'emb_size': 100,
    'out_channels': 10,
    'num_classes': 1,
    'load_embeddings': True,
    'rnn_embeddings': False,
    'freeze_embeddings': True,
    'emb_path': '/Users/alexander/Documents/jigsaw/vectors/fast_text/model_100_5.bin',
    'save_path': '../models/linear/fasttext/fasttext.joblib',
    'sample_submission': '../data/jigsaw-rate-severity/sample_submission.csv',
    'output_dir': '.'
}
cfg = Box(cfg)
# t = Tokenizer()
# t.fit(train_df)
# cfg['tokenizer'] = t

In [58]:
def tfifdf_preprocess(data, col):
    data[col] = links_removing(data, col)
    data[col] = remove_stopwords(data, col)
    data[col] = preprocess_from_kaggle(data, col)
    data[col] = preprocess_slang_sub(data, col)
    data[col] = lemmatize(data, col)
    data[col] = remove_punctuation(data, col)
    data[col] = data[col].apply(lambda x: replaceMultiToxicWords(x))
    return data

In [59]:
def preprocess_train(df):
    toxic_comment_w = {
        'identity_hate': 1.18323494807095,
        'insult': 0.3844723322363254,
        'obscene': 1.241559480304455,
        'severe_toxic': 1.147691258564159,
        'threat': 1.273216379557446,
        'toxic': 1.966945011013778
    }
    
    for col, v in toxic_comment_w.items():
        df[col] *= toxic_comment_w[col]
        df['y'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)
        df['y'] = df['y']/df['y'].max()
    return df

In [60]:
def clean_val(df):
  df_new = df.copy()
  df_new['pair'] = df_new.apply(lambda x:" ".join(sorted((x['less_toxic'],
                                                  x['more_toxic']))),axis=1)
  df_new['pair_hash'] = df_new.pair.apply(lambda x: str(abs(hash(x)) % (10 ** 8)))
  del df_new['pair']

  df_new['pair_cnt']=df_new.groupby(['pair_hash'])['worker'].transform(lambda x: x.count())

  df_new['cnt']=df_new.groupby(['pair_hash', 
                        'less_toxic',
                        'more_toxic'])['worker'].transform(lambda x: x.count())
  df_new = df_new[~((df_new.pair_cnt == 3) & (df_new.cnt == 1))][['worker', 'less_toxic', 'more_toxic']]
  df_new = df_new.drop_duplicates(subset = ['less_toxic', 'more_toxic'])
  df_new.index = range(len(df_new))
  return df_new

In [61]:
df = pd.read_csv('../data/jigsaw-rate-severity/validation_data.csv')
df = clean_val(df)

In [62]:
train_df = pd.read_csv('../data/preprocessed_data/jigsaw-toxic-comment_lemmatize.csv')
val_df = pd.read_csv('../data/preprocessed_data/validation_data_lemmatize.csv')
test_df = pd.read_csv('../data/jigsaw-rate-severity/comments_to_score.csv')

In [39]:
linear_train(cfg, train_df, val_df)

Global seed set to 42


wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Adding directory to artifact (./../models/linear/fasttext)... Done. 0.0s


loading features from cache
loading features from cache
loading features from cache


val_acc,▁


0.7051256679200475

In [ ]:
#0.677
#0.69

In [ ]:
# best_glove: 840B_300d
# best_fasttext: model_300_5

In [44]:
import lightgbm
from jigsaw.utils.fast_text import convert_fasttext_to_features
from jigsaw.utils.glove import load_glove, convert_glove_to_features

In [63]:
# emmbed_dict = load_glove(cfg.emb_path)
X = convert_fasttext_to_features(train_df['comment_text'], cfg.emb_path)
# X = convert_glove_to_features(train_df['comment_text'], emmbed_dict)

converting fasttext to features:   0%|          | 0/159534 [00:00<?, ?it/s]

In [64]:
X_less = convert_fasttext_to_features(
    val_df['less_toxic'],
    cfg.emb_path
)

X_more = convert_fasttext_to_features(
    val_df['more_toxic'],
    cfg.emb_path
)

converting fasttext to features:   0%|          | 0/10106 [00:00<?, ?it/s]

converting fasttext to features:   0%|          | 0/10106 [00:00<?, ?it/s]

In [76]:
reg = lightgbm.LGBMRegressor(
    n_estimators = 350,
    learning_rate = 0.1
)
reg.fit(X, train_df['y'].values*100)
less_toxic_y = reg.predict(X_less)
more_toxic_y = reg.predict(X_more)
(more_toxic_y > less_toxic_y).mean()

0.7225410647140312

In [66]:
!mkdir ../models/gbm/fasttext

In [67]:
import joblib
# save model
joblib.dump(reg, '../models/gbm/fasttext/lgb.pkl')

['../models/gbm/fasttext/lgb.pkl']

In [68]:
m = joblib.load('../models/gbm/fasttext/lgb.pkl')

In [81]:
(train_df['y'].values*100).max()

100.0

In [83]:
reg.predict(X_less).min()

-3.96228451187367

In [ ]:
0.7292
0.7308, 350, 0.1
0.7337, 400, 0.1
0.7342, 450, 0.1

In [84]:
import nltk

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [1]:
ls

CNN_train.ipynb      RNN_train.ipynb      tokenizer.pickle
Dataset_gen.ipynb    RoBerta_train.ipynb  tokenizer_unitary/
EDA.ipynb            lightning_logs/      wandb/
Linear.ipynb         models/


In [2]:
import requests

In [12]:
r = requests.get('http://localhost:8080/api/data/test')

In [13]:
r.headers

{'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Length': '0', 'Date': 'Tue, 08 Feb 2022 12:27:19 GMT', 'Keep-Alive': 'timeout=60', 'Connection': 'keep-alive'}